<a href="https://colab.research.google.com/github/ninad-1234/Web_Minning/blob/main/whoosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install Whoosh
# installing Whoosh

     |████████████████████████████████| 471kB 5.0MB/s 


In [9]:
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED,NUMERIC
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer
from whoosh import index
import os, os.path
# imported Required Packages 

Defined the schema of the document :- 

In [10]:
schema = Schema(year=NUMERIC(stored=True),
                author=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                title=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                abstract=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                body=TEXT(analyzer=StandardAnalyzer(stoplist=None)),
                subject=KEYWORD(commas=True,scorable=True),
                keywords=KEYWORD(commas=True, scorable=True))

uploading the document in proper schema 

In [11]:

if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = index.create_in("indexdir", schema)

ix = index.open_dir("indexdir")

writer = ix.writer()


abstract1=u'''It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson'''

abstract2=u'''Children's and Household Tales (German: Kinder- und Hausmärchen) is a collection of fairy tales first published in 20 December 1812 by the Grimm brothers, Jacob and Wilhelm. The collection is commonly known in English as Grimms' Fairy Tales.'''

writer.add_document(year=u"1936",
                author=u"MargareJ",
                title=u"Gone with the wind",
                abstract=abstract1,
                subject=u"novel, love",
                keywords=u"Scarlett, Rhett")
writer.add_document(year=u"1812",
                author=u" Jacob and Wilhelm",
                title=u"Grimms' Fairy Tales",
                abstract=abstract2,
                subject=u"story, children",
                keywords=u"The Frog King,  Rapunzel")
 
writer.commit()

Examples of Parsing

In [12]:

parser=QueryParser("abstract",schema=schema) 

result=parser.parse(u"Samsung company department")
print result

(abstract:samsung AND abstract:company AND abstract:department)


In [13]:

result=parser.parse(u"apple company department")
print result

(abstract:apple AND abstract:company AND abstract:department)


Advance Parsing Example:- 

In [14]:

parser.add_plugin(GtLtPlugin()) 
result=parser.parse(u"year:<2000")
print result

year:[ TO 2000}


Applying search engine for mentioned documents :-

In [15]:

parser.add_plugin(FuzzyTermPlugin())
result=parser.parse(u"author:margare~")
print result

with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results[0])
    #print(results)

author:margare~
<Hit {'title': u'Gone with the wind', 'abstract': u"It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson", 'author': u'MargareJ', 'year': u'1936'}>


In [16]:

result=parser.parse(u'title:"gonE WITH"~2')
print result
with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)

title:"gone with"
<Top 1 Results for Phrase(u'title', [u'gone', u'with'], slop=2, boost=1.000000) runtime=0.00109791755676>


In [17]:

result=parser.parse(u'title:go*')
print result
with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)
    print (results[0])

title:go*
<Top 1 Results for Prefix(u'title', u'go') runtime=0.000636100769043>
<Hit {'title': u'Gone with the wind', 'abstract': u"It depicts the struggles of young Scarlett O'Hara, the spoiled daughter of a well-to-do plantation owner, who must use every means at her disposal to claw her way out of poverty following Sherman's destructive 'March to the Sea'. This historical novel features a Bildungsroman or coming-of-age story, with the title taken from a poem written by Ernest Dowson", 'author': u'MargareJ', 'year': u'1936'}>


In [18]:

parser.remove_plugin_class(PhrasePlugin)

parser.add_plugin(SequencePlugin())

result=parser.parse(u'"abstract:(child OR childr*) abstract:ho*sehold"~3 AND title:tale*')
print (result)
with ix.searcher() as searcher:
    results=searcher.search(result)
    print (results)
    
    import re
    for result in results:
        analyzer=StandardAnalyzer(stoplist=None)
        a=[(t.pos) for t in analyzer(result['abstract'],positions=True) if re.match(r"tale*",t.text)]
        print "the position of the word pattern "+"<tale*> "+"in document <"+result['title']+"> is:"
        print a

(((abstract:child OR abstract:childr*) NEAR abstract:ho*sehold) AND title:tale*)
<Top 1 Results for And([Sequence([Or([Term(u'abstract', u'child'), Prefix(u'abstract', u'childr')]), Wildcard(u'abstract', u'ho*sehold')], slop=3, boost=1.000000), Prefix(u'title', u'tale')]) runtime=0.00353193283081>
the position of the word pattern <tale*> in document <Grimms' Fairy Tales> is:
[4, 14, 38]
